In [1]:
import os
import pandas as pd
import random

# 원본 데이터
data = {
    "도수": ["낮은", "중간", "높은", "알 수 없음"],
    "술 종류": ["칵테일", "보드카", "위스키", "럼", "알 수 없음"],
    "맛": ["달달한", "쓴맛", "부드러운", "상큼한", "신맛", "알 수 없음"]
}

# 동의어 사전 (기존과 동일)
flavor_variants = {
    "달달한": ["달콤한", "스위트한"],
    "쓴맛": ["쌉쌀한", "비터한"],
    "부드러운": ["매끄러운", "소프트한"],
    "상큼한": ["산뜻한", "프레시한"],
    "신맛": ["톡 쏘는", "상큼한"]
}

# 새로운 문장 패턴 (부분 정보만 있는 경우)
partial_patterns = [
    # 도수만 언급
    "도수가 {도수}인 술을 추천해줘.",
    "{도수} 도수의 술을 찾고 있어.",
    "도수는 {도수}로 부탁해요.",
    
    # 맛만 언급
    "{맛} 술을 마시고 싶어.",
    "{맛} 맛이 나는 술을 추천해줘.",
    "{맛}한 술 없을까요?",
    
    # 종류만 언급
    "{술_종류}를 마시고 싶어.",
    "{술_종류} 추천해줄래?",
    "오늘은 {술_종류} 어때요?",
    
    # 도수 + 맛
    "{도수} 도수의 {맛} 술을 찾아줘.",
    "{맛}하고 도수는 {도수}인 술 추천해줘.",
    
    # 도수 + 종류
    "{도수} 도수의 {술_종류}를 마시고 싶어.",
    "{술_종류} 중에서 {도수} 도수 추천해줘.",
    
    # 맛 + 종류
    "{맛} {술_종류}를 추천해줘.",
    "{술_종류} 중에서 {맛}한 걸로 부탁해요."
]

# 문장 패턴
full_patterns = [
    "나는 {도수} 도수의 {맛} {술_종류}을 마시고 싶어.",
    "{맛} {술_종류} 중에서 {도수} 도수의 것을 추천해줘.",
    "오늘 {맛} {술_종류}를 마시고 싶어, 도수는 {도수}로.",
    "{도수} 도수의 {맛} {술_종류} 한 잔 어때?",
    "지금 {맛} {술_종류}가 마시고 싶은데, {도수} 도수가 좋아.",
    "{도수} 도수로 {맛} {술_종류}를 마셔보고 싶어요.",
    "{맛} {술_종류} 중에서 {도수} 도수가 괜찮을까요?",
    "{도수} 도수의 {맛} {술_종류}, 추천할 만한 거 있어?",
    "마실 {술_종류}를 찾고 있어요. {맛}이고, {도수} 도수면 좋겠어요.",
    "도수가 {도수}인 {맛} {술_종류}를 마셔보고 싶어요.",
    "{맛} {술_종류} 중에서 도수 {도수}인 게 궁금해요.",
    "{도수} 도수의 {술_종류}를 추천해주세요, {맛}으로!"
    "오늘은 {도수} 도수의 {맛} {술_종류}를 마셔야겠어."
    "{술_종류}를 마시고 싶어요. {맛}이고 도수는 {도수}가 좋아요."
    "{맛} {술_종류} 한 잔 어때? 도수는 {도수}로!"
    "가볍게 마실 수 있는 {도수} 도수의 {맛} {술_종류}를 찾고 있어요."
    "{도수} 도수의 {술_종류} 중에 {맛}인 걸 추천해주세요."
    "친구랑 마실 {맛} {술_종류}, {도수} 도수로 부탁해요."
    "도수 {도수}인 {맛} {술_종류}를 마셔보고 싶은 기분이에요."
    "{맛} {술_종류}가 필요해요. 도수는 {도수}가 좋겠어요."
    "오늘의 추천 {술_종류}는 {도수} 도수의 {맛} 걸로 해주세요."
    "{맛} {술_종류} 중에서 도수가 {도수}인 게 궁금하네요."
    "{도수} 도수로 마실 만한 {맛} {술_종류}, 알려줘."
    "요즘 {도수} 도수의 {맛} {술_종류}에 빠졌어요."
    "도수가 {도수}이고 {맛} {술_종류}, 괜찮을까요?"
    "{맛} {술_종류}가 끌리는데, 도수는 {도수}로 골라줘."
    "{도수} 도수의 {맛} {술_종류} 한 잔 생각나네요."
    "{술_종류} 추천해줘! {맛}이고 도수는 {도수}로."
]

# 부사 추가
adverbs = ["정말", "매우", "조금", "꽤", "아주", "상당히"]

def augment_data(data, num_samples=1000):
    augmented_data = {"입력 문장": [], "도수": [], "술 종류": [], "맛": []}
    
    for _ in range(num_samples):
        # 각 속성을 포함할 확률 설정
        include_proof = random.random() < 0.8  # 80% 확률로 도수 포함
        include_type = random.random() < 0.8   # 80% 확률로 종류 포함
        include_taste = random.random() < 0.8  # 80% 확률로 맛 포함
        
        # 최소한 하나는 포함되도록 보장
        if not any([include_proof, include_type, include_taste]):
            choice = random.choice(['proof', 'type', 'taste'])
            if choice == 'proof': include_proof = True
            elif choice == 'type': include_type = True
            else: include_taste = True
        
        # 값 선택
        도수 = random.choice(data["도수"][:-1]) if include_proof else "알 수 없음"
        술_종류 = random.choice(data["술 종류"][:-1]) if include_type else "알 수 없음"
        맛_original = random.choice(data["맛"][:-1]) if include_taste else "알 수 없음"
        
        # 패턴 선택
        if all([include_proof, include_type, include_taste]):
            pattern = random.choice(full_patterns)
            맛 = random.choice(flavor_variants.get(맛_original, [맛_original]))
            # 부사 추가 (30% 확률)
            if random.random() < 0.3:
                맛 = f"{random.choice(adverbs)} {맛}"
        else:
            pattern = random.choice(partial_patterns)
            맛 = 맛_original
        
        # 문장 생성
        format_args = {}
        if include_proof: format_args['도수'] = 도수
        if include_type: format_args['술_종류'] = 술_종류
        if include_taste: format_args['맛'] = 맛
        
        try:
            input_sentence = pattern.format(**format_args)
            
            augmented_data["입력 문장"].append(input_sentence)
            augmented_data["도수"].append(도수)
            augmented_data["술 종류"].append(술_종류)
            augmented_data["맛"].append(맛_original)
        except KeyError:
            continue
    
    return augmented_data

# 데이터 생성 및 저장
augmented_data = augment_data(data, num_samples=2000)  # 샘플 수 증가
df_augmented = pd.DataFrame(augmented_data)

output_dir = "data"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "augmented_data.csv")
df_augmented.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"총 {len(df_augmented)}개의 증강된 데이터가 '{output_path}'에 저장되었습니다!")

총 1493개의 증강된 데이터가 'data\augmented_data.csv'에 저장되었습니다!
